# Leaf disk sheat segmentation and disk coordonates detection with SegFormer

## Imports

In [ ]:
import os
import sys
from pathlib import Path
import datetime

from collections import OrderedDict

from tqdm.notebook import tqdm

%load_ext autoreload
%autoreload 2

from collections import defaultdict
import copy
import random

import pandas as pd

from sklearn.model_selection import train_test_split

import albumentations as A

from PIL import Image

from sklearn.model_selection import train_test_split

from albumentations.pytorch import ToTensorV2

import cv2
import matplotlib.pyplot as plt
import numpy as np

import ternausnet.models
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
import torchvision

from pytorch_lightning.callbacks import TQDMProgressBar
from pytorch_lightning.callbacks import RichProgressBar

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import DeviceStatsMonitor
from pytorch_lightning.callbacks import ModelCheckpoint

import pytorch_lightning as pl

import ipywidgets as widgets
from IPython.display import Image as IpImage
from IPython.display import display
from ipywidgets import Button, HBox, VBox

sys.path.insert(0, os.path.join("..", "scripts"))

import ld_dataset as ldd
import ld_plot as ldp
import ld_image as ldi
import ld_th_pl_lightning as ldpl
import gav_oidium_const as goc


## Build local images dataframe

In [ ]:
una_images = [i.stem for i in ldd.una_images_folder.glob("*") if not i.stem.startswith(".")]
ano_images = [i.stem for i in ldd.train_images_folder.glob("*") if not i.stem.startswith(".")]
wot_images = [i.stem for i in ldd.wot_images_folder.glob("*") if not i.stem.startswith(".")]


In [ ]:
len(ano_images), len(una_images), len(wot_images)

In [ ]:
local_images = list(set(ano_images + una_images + wot_images))
len(local_images)

In [ ]:
all_columns = [
    "experiment",
    "rep",
    "image_name",
    "ligne",
    "colonne",
    "oiv",
    "sporulation",
    "densite_sporulation",
    "necrose",
    "surface_necrosee",
    "taille_necrose",
]

df = (
    pd.read_csv(os.path.join("..", "data_in", "raw_merged.csv"), sep=";")
)
df

In [ ]:
df = df[df.image_name.str.lower().replace("_", "-").isin([l.lower().replace("_", "-") for l in local_images])]

df.sort_values("image_name").to_csv(
    os.path.join("..", "data_in", "local_raw_merged.csv"),
    sep=";",
    index=False,
)

df


## Prepare envionment

In [ ]:
ldpl.g_device


In [ ]:
sample_frac = 1

## Check image duplicates

In [ ]:
una_images = [str(i) for i in ldd.una_images_folder.glob("*") if not i.stem.startswith(".")]
ano_images = [str(i) for i in ldd.train_images_folder.glob("*") if not i.stem.startswith(".")]

len(una_images), len(ano_images)

#### Check anotated images in not annotated images

In [ ]:
[img for img in ano_images if img in una_images]

#### Check not anotated images in annotated images

In [ ]:
[img for img in una_images if img in ano_images]

## Images

In [ ]:
df_train_images = ldd.build_items_dataframe(
    images_folder=ldd.train_images_folder,
    masks_folder=ldd.train_masks_folder,
)

In [ ]:
df_train_images

In [ ]:
ldd.check_items_consistency(
    images_folder=ldd.train_images_folder,
    masks_folder=ldd.train_masks_folder,
)

In [ ]:
train, test = train_test_split(df_train_images, test_size=0.3, stratify=df_train_images["year"])
test, val = train_test_split(test, test_size=0.5, stratify=test["year"])

print(len(train), len(test), len(val))


In [ ]:
df_set_counts = pd.DataFrame(
    index=df_train_images.year.sort_values().value_counts(sort=False).index,
    data={
        "train": train.year.sort_values().value_counts(sort=False).values,
        "val": val.year.sort_values().value_counts(sort=False).values,
        "test": test.year.sort_values().value_counts(sort=False).values,
    },
)
df_set_counts


In [ ]:
df_set_counts.sum(axis=0)

In [ ]:
ldp.display_images_and_masks_grid(train.sample(n=2), fontsize=10, figsize=(8,6))


In [ ]:
ldp.display_images_and_masks_grid(val.sample(n=2), fontsize=10, figsize=(8,6))


In [ ]:
ldp.display_images_and_masks_grid(test.sample(n=2), fontsize=10, figsize=(8,6))


## Albumentations

In [ ]:
img_width = 32 * 21
img_height = 32 * 14

assert(img_width / img_height == 1.5)

alb_resizer = [A.Resize(height=img_height, width=img_width)]

train_transformers_list = alb_resizer + [
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
    A.RandomGamma(p=0.33),
    A.CLAHE(p=0.33),
]

to_tensor = [
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
]

transformer = A.Compose(train_transformers_list)

image, mask = ldd.open_image_and_mask(
    0, df_train_images.sample(n=1).reset_index(drop=True)
)


ldp.visualize_augmentations(
    image=image,
    mask=mask,
    augmentations=[transformer(image=image, mask=mask) for _ in range(5)],
)

## Dataset

In [ ]:
from torch.utils.data import Dataset



def get_mask_path(image_name: str, df: pd.DataFrame):
    try:
        mask_path = df[df.image_name == image_name].mask_path.to_list()[0]
    except:
        return None
    else:
        return mask_path


def get_image_path(image_name, df: pd.DataFrame):
    try:
        image_path = df[df.image_name == image_name].image_path.to_list()[0]
    except:
        return None
    else:
        return image_path


def get_image_name_at(idx: int, df: pd.DataFrame) -> str:
    return df.image_name.to_list()[0]


def open_mask(path, df: pd.DataFrame = None):
    mask_path = path if df is None else get_mask_path(image_name=path, df=df)
    if mask_path is not None:
        # mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE).astype(np.float32)
        # mask[mask == 0.0] = 0.0
        # mask[mask != 0.0] = 1.0
        # return mask
        return Image.open(mask_path)
    else:
        return None


def open_image(path, df: pd.DataFrame = None):
    return Image.open(path if df is None else get_image_path(image_name=path, df=df))
    # return cv2.cvtColor(
    #     cv2.imread(path if df is None else get_image_path(image_name=path, df=df)),
    #     cv2.COLOR_BGR2RGB,
    # )


def open_image_and_mask(key, df: pd.DataFrame):
    if isinstance(key, str):
        return open_image(key, df), open_mask(key, df)
    if isinstance(key, int):
        filename = get_image_name_at(key, df)
        return open_image(filename, df), open_mask(filename, df)
    return None, None


class SemanticSegmentationDataset(Dataset):
    """Image (semantic) segmentation dataset."""

    def __init__(self, df_img, feature_extractor, augmenter):
        self.df_img = df_img
        self.augmenter = augmenter
        self.feature_extractor = feature_extractor

    def __len__(self):
        return self.df_img.shape[0]

    def __getitem__(self, idx):
        image, segmentation_map = open_image_and_mask(key=idx, df=self.df_img)
        # if self.augmenter is not None:
        #     augmented = self.augmenter(image=image, mask=segmentation_map)
        #     image = augmented["image"]
        #     segmentation_map = augmented["mask"]

        # image = np.moveaxis(image, 2, 0)

        # randomly crop + pad both image and segmentation map to same size
        encoded_inputs = self.feature_extractor(
            image, segmentation_map, return_tensors="pt"
        )

        for k, v in encoded_inputs.items():
            encoded_inputs[k].squeeze_()  # remove batch dimension

            return encoded_inputs


In [ ]:
from transformers import SegformerFeatureExtractor

feature_extractor = SegformerFeatureExtractor(reduce_labels=True, do_resize=False)

train_dataset = SemanticSegmentationDataset(
    df_img=train,
    feature_extractor=feature_extractor,
    augmenter=A.Compose(train_transformers_list),
)
valid_dataset = SemanticSegmentationDataset(
    df_img=val,
    feature_extractor=feature_extractor,
    augmenter=A.Compose(alb_resizer),
)


In [ ]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(valid_dataset))

In [ ]:
encoded_inputs = train_dataset[0]

In [ ]:
encoded_inputs["pixel_values"].shape

In [ ]:
encoded_inputs["labels"].shape

In [ ]:
encoded_inputs["labels"]

In [ ]:
encoded_inputs["labels"].squeeze().unique()

## Data loaders

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=2)

In [ ]:
batch = next(iter(train_dataloader))

In [ ]:
for k,v in batch.items():
  print(k, v.shape)

In [ ]:
batch["labels"].shape

In [ ]:
mask = (batch["labels"] != 255)
mask

In [ ]:
batch["labels"][mask]

## The model

In [ ]:
from transformers import SegformerForSemanticSegmentation
import json
from huggingface_hub import cached_download, hf_hub_url

# load id2label mapping from a JSON on the hub

id2label = {0:"background", 255:"leaf disk"}
label2id = {v: k for k, v in id2label.items()}

# define model
model = SegformerForSemanticSegmentation.from_pretrained(
    "nvidia/mit-b0",
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
)


## Finetune

In [ ]:
from datasets import load_metric

metric = load_metric("mean_iou")

In [ ]:
import torch
from torch import nn
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm

# define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006)
# move model to GPU
device = "cpu"#ldpl.g_device
model.to(device)

model.train()
for epoch in range(200):  # loop over the dataset multiple times
   print("Epoch:", epoch)
   for idx, batch in enumerate(tqdm(train_dataloader)):
        # get the inputs;
        pixel_values = batch["pixel_values"].type(torch.float).to(device)
        labels = batch["labels"].type(torch.float).to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss, logits = outputs.loss, outputs.logits
        
        loss.backward()
        optimizer.step()

        # evaluate
        with torch.no_grad():
          upsampled_logits = nn.functional.interpolate(logits, size=labels.shape[-2:], mode="bilinear", align_corners=False)
          predicted = upsampled_logits.argmax(dim=1)
          
          # note that the metric expects predictions + labels as numpy arrays
          metric.add_batch(predictions=predicted.detach().cpu().numpy(), references=labels.detach().cpu().numpy())

        # let's print loss and metrics every 100 batches
        if idx % 100 == 0:
          metrics = metric._compute(num_labels=len(id2label), 
                                   ignore_index=255,
                                   reduce_labels=False, # we've already reduced the labels before)
          )

          print("Loss:", loss.item())
          print("Mean_iou:", metrics["mean_iou"])
          print("Mean accuracy:", metrics["mean_accuracy"])

## Models

### Lightning

In [ ]:
img_width = 32 * 21
img_height = 32 * 14

assert(img_width / img_height == 1.5)

alb_resizer = [A.Resize(height=img_height, width=img_width)]

train_transformers_list = alb_resizer + [
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
    A.RandomGamma(p=0.33),
    A.CLAHE(p=0.33),
]

to_tensor = [
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
]

model = ldpl.LeafDiskSegmentation(
    batch_size=16,
    selected_device=ldpl.g_device,
    learning_rate=0.001,
    max_epochs=400,
    num_workers=0,
    train_augmentations=A.Compose(train_transformers_list + to_tensor),
    train_data=train,
    val_augmentations=A.Compose(alb_resizer + to_tensor),
    val_data=val,
    accumulate_grad_batches=3,
)

trainer = Trainer(
    accelerator="gpu",
    max_epochs=model.max_epochs,
    log_every_n_steps=5,
    callbacks=[
        RichProgressBar(),
        EarlyStopping(monitor="val_loss", mode="min", patience=15, min_delta=0.0005),
        DeviceStatsMonitor(),
        ModelCheckpoint(
            save_top_k=3,
            monitor="val_loss",
            auto_insert_metric_name=True,
            filename="{epoch}-{step}-{train_loss:.5}-{val_loss:.5f}",
        ),
    ],
    accumulate_grad_batches=model.accumulate_grad_batches,
)
trainer.fit(model)

In [ ]:
version_overview = ldpl.update_overviews(test).sort_values(["test_loss"])
version_overview


In [ ]:
predictor = ldpl.LeafDiskSegmentationPredictor()

op_df_versions_overview = widgets.Output()
with op_df_versions_overview:
    display(
        version_overview.drop(["checkpoint_fileName"], axis=1).reset_index(drop=True)
    )

cb_select_version = widgets.Dropdown(
    options=["-1 | Select version"]
    + [
        f"{i} | {fn}"
        for i, fn in enumerate(version_overview.checkpoint_fileName.to_list())
    ],
    description="Select version",
)

cb_image = widgets.Dropdown(
    options=sorted([str(i) for i in ldd.una_images_folder.glob("*") if not i.stem.startswith(".")]),
    description="Select an image:",
)

cb_ancillary_image = widgets.Dropdown(
    options=["Raw mask", "Cleaned mask", "Probabilities"],
    description="Ancillary image",
)

src_image = widgets.Output()
ancillary_image = widgets.Output()
pred_image = widgets.Output()


def update_images(image_path, anc_mode):
    if predictor.model is None:
        src_image.clear_output()
        ancillary_image.clear_output()
        pred_image.clear_output()
        return

    image = ldd.open_image(image_path)

    predicted_mask = predictor.predict_image(image_path)
    clean_mask = ldi.clean_contours(mask=predicted_mask.copy(), size_thrshold=0.75)
    contours = ldi.index_contours(clean_mask)

    anc_img = (
        predicted_mask
        if anc_mode == "Raw mask"
        else clean_mask
        if anc_mode == "Cleaned mask"
        else predictor.predict_image(image_path, return_probabilities=True)
        if anc_mode == "Probabilities"
        else None
    )

    src_image.clear_output()
    with src_image:
        ldp.visualize_image(
            image=image,
            title=Path(image_path).stem,
        )

    ancillary_image.clear_output()
    with ancillary_image:
        plt.imshow(anc_img, cmap=plt.cm.RdPu)
        plt.tight_layout()
        plt.axis("off")
        plt.show()

    pred_image.clear_output()
    with pred_image:
        ldp.visualize_image(
            ldi.print_contours_indexes(
                clean_mask,
                contours,
                canvas=ldi.apply_mask(image, clean_mask, draw_contours=8),
            ),
            figsize=(12, 8),
        )


def on_image_changed(change):
    update_images(change.new, cb_ancillary_image.value)


def on_anc_mode_changed(change):
    update_images(cb_image.value, change.new)


def on_version_changed(change):
    idx, filename = str(change.new).replace(" ", "").split("|")
    if int(idx) >= 0:
        global predictor
        predictor.model = ldpl.LeafDiskSegmentation.load_from_checkpoint(filename)
        update_images(cb_image.value, cb_ancillary_image.value)
    else:
        predictor.model = None


cb_select_version.observe(on_version_changed, names="value")
cb_image.observe(on_image_changed, names="value")
cb_ancillary_image.observe(on_anc_mode_changed, names="value")

display(
    VBox(
        [
            HBox([op_df_versions_overview, cb_select_version]),
            HBox([cb_image, cb_ancillary_image]),
            HBox([VBox([src_image, ancillary_image]), pred_image]),
        ]
    )
)


In [ ]:
str(cb_select_version.value).replace(" ", "").split("|")[1]

## Test all available not used images

Create the target folder

In [ ]:
data_out_fld = ldd.root_folder.parent.parent.joinpath("data_out", "predictions", datetime.datetime.now().strftime('%Y%m%d%H%M%S'))
data_out_fld.mkdir(parents=True, exist_ok=True)
data_out_fld

In [ ]:
for i in tqdm(ldd.una_images_folder.glob("*")):
    if i.stem.startswith("."):
        continue

    image = ldd.open_image(str(i))
    predicted_mask = predictor.predict_image(str(i))
    clean_mask = ldi.clean_contours(mask=predicted_mask.copy(), size_thrshold=0.75)
    contours = ldi.index_contours(clean_mask)

    image = Image.fromarray(
        ldi.print_contours_indexes(
            clean_mask,
            contours,
            canvas=ldi.apply_mask(image, clean_mask, draw_contours=8),
        )
    )
    image.save(str(data_out_fld.joinpath(i.name)))


## Retrieve isolated leaf disk

In [ ]:
cb_src_image = widgets.Dropdown(
    options=sorted(
        [str(i) for i in ldd.una_images_folder.glob("*") if not i.stem.startswith(".")]
    ),
    description="Select a leaf dislk:",
)
cb_row = widgets.Dropdown(options=["A", "B", "C"])
cb_col = widgets.Dropdown(options=[1, 2, 3, 4])

op_full_plate = widgets.Output()
op_leafdisk = widgets.Output()
op_leafdisk_no_bck = widgets.Output()


def show_leaf_disk(image_path, row, col):
    image = ldd.open_image(image_path)

    predicted_mask = predictor.predict_image(image_path)
    clean_mask = ldi.clean_contours(mask=predicted_mask.copy(), size_thrshold=0.75)
    contours = ldi.index_contours(clean_mask)

    op_full_plate.clear_output()
    op_leafdisk.clear_output()
    op_leafdisk_no_bck.clear_output()

    with op_full_plate:
        ldp.visualize_image(
            ldi.print_single_contour(
                clean_mask,
                contours,
                row=row,
                col=col,
                canvas=ldi.apply_mask(image, clean_mask, draw_contours=8),
            ),
            figsize=(12, 8),
        )

    with op_leafdisk:
        ldp.visualize_image(ldi.get_leaf_disk(image.copy(), contours, row, col))

    with op_leafdisk_no_bck:
        ldp.visualize_image(
            ldi.get_leaf_disk(
                image.copy(),
                contours,
                row,
                col,
                mask=clean_mask,
            )
        )


def on_ld_image_changed(change):
    show_leaf_disk(change.new, cb_row.value, cb_col.value)


def on_ld_row_changed(change):
    show_leaf_disk(cb_src_image.value, change.new, cb_col.value)


def on_ld_col_changed(change):
    show_leaf_disk(cb_src_image.value, cb_row.value, change.new)


cb_src_image.observe(on_ld_col_changed, names="value")
cb_row.observe(on_ld_row_changed, names="value")
cb_col.observe(on_ld_col_changed, names="value")

display(
    VBox(
        [
            HBox([cb_src_image, cb_row, cb_col]),
            HBox([op_full_plate]),
            HBox([op_leafdisk_no_bck, op_leafdisk]),
        ]
    )
)
